### 인스톨

In [5]:
!pip install keras
!pip install tensorflow
!pip install opencv-python
!pip install numpy==1.16.2
# !pip install numpy==1.16.2 --user

  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] 액세스가 거부되었습니다: 'c:\\programdata\\anaconda3\\lib\\site-packages\\numpy\\compat\\py3k.py'
Consider using the `--user` option or check the permissions.



### 모델 이름

In [1]:
model_name = "model04-0.3338.hdf5"

### 임포트

In [2]:
from socket import *
import os, re, glob
import numpy as np
import pandas as pd
import shutil
from numpy import argmax
from keras.models import load_model

import konlpy
from konlpy.tag import Okt
okt = Okt()

from keras.preprocessing.text import Tokenizer

from tqdm import tqdm_notebook

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [13]:
np.version.version

'1.16.2'

### 서버 설정부

In [3]:
serverSock = socket(AF_INET, SOCK_STREAM)
serverSock.bind(('', 5555))
serverSock.listen(1)

OSError: [WinError 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다

### 모델 로드부

In [4]:
target_model = load_model('./model/' + model_name) #모델은 폴더가서 앞숫자가 가장 높은거 확인하구 복붙하귀







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
target_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 100)         3500000   
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 3,617,377
Trainable params: 3,617,377
Non-trainable params: 0
_________________________________________________________________


### 설정 (통과)

In [3]:
train_data = pd.read_table('data/ratings_train.txt')
test_data = pd.read_table("data/ratings_test.txt")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [26]:
data1 = pd.read_csv("data/export.csv",encoding="cp949")

pandas.core.frame.DataFrame

In [29]:
train_data = pd.concat([train_data,data1])

In [30]:
train_data=train_data.dropna(how='any') # Null 값이 존재하는 행 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")# 한글과 공백을 제외하고 모두 제거

In [31]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
#불용어 

In [32]:
X_train=[]
for sentence in train_data['document']:
    temp_X = []
    temp_X=okt.morphs(sentence, stem=True) # 토큰화
    temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [34]:
test_data=test_data.dropna(how='any') # Null 값 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행

X_test=[]
for sentence in tqdm_notebook(test_data['document']):
    temp_X = []
    temp_X=okt.morphs(sentence, stem=True) # 토큰화
    temp_X=[word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [36]:
y_train=train_data['label']
y_test=test_data['label']

In [37]:
np.save('data/Train_test_final2.npy',(X_train,X_test,y_train,y_test))

### 여기서부터!

In [10]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
#불용어 

In [11]:
X_train, X_test ,y_train ,y_test = np.load("data/Train_test_final2.npy")

In [12]:
max_words = 35000
tokenizer = Tokenizer(num_words=max_words) # 상위 35,000개의 단어만 보존
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [29]:
def pos_neg(question):
    result=[]

    temp = 'question'.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    temp = okt.morphs(question, stem=True)
    temp =[word for word in temp if not word in stopwords] # 불용어 제거

    result.append(temp)
    result = tokenizer.texts_to_sequences(result)

    score = target_model.predict(result).mean()
    
    msg = int(score*10)
    if(msg == 0):
        msg = 1
    
    return str(msg)

### 서버구동부

In [46]:
data = "start"

try:
    while data != "end" :
        connectionSock, addr = serverSock.accept()
        print(str(addr),'에서 접속이 확인되었습니다.')

        data = connectionSock.recv(1024).decode('utf-8')
        print('받은 데이터 : ', data)
#####################################분류 결정부####################################
        result_data = pos_neg(str(data))
        print('분류 데이터 : ', result_data)
        result = result_data +'\n'
#####################################################################################
        connectionSock.send(result.encode('utf-8'))
        print('결과를 전송했습니다. \n')
except Exception as e:
    print(e)
    
print("########################분류서버종료########################")

('127.0.0.1', 1557) 에서 접속이 확인되었습니다.
받은 데이터 :  개새끼야!
분류 데이터 :  2
결과를 전송했습니다. 

('127.0.0.1', 1567) 에서 접속이 확인되었습니다.
받은 데이터 :  창녀야!
분류 데이터 :  2
결과를 전송했습니다. 

('127.0.0.1', 1586) 에서 접속이 확인되었습니다.
받은 데이터 :  end
'list' object has no attribute 'mean'
########################분류서버종료########################
